In [5]:
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
import altair

%matplotlib inline
plt.rcParams["patch.force_edgecolor"] = True
%store -r cleaned

Installing plotly

In [8]:
import sys
!/opt/anaconda/bin/conda install --yes --prefix {sys.prefix} plotly

Solving environment: done

## Package Plan ##

  environment location: /home/toni/.conda/envs/digital_divide

  added / updated specs: 
    - plotly


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    retrying-1.3.3             |           py36_2          15 KB
    plotly-3.1.0               |           py36_0        27.4 MB
    openssl-1.0.2o             |       h14c3975_1         3.4 MB
    ------------------------------------------------------------
                                           Total:        30.8 MB

The following NEW packages will be INSTALLED:

    plotly:          3.1.0-py36_0              
    retrying:        1.3.3-py36_2              

The following packages will be UPDATED:

    ca-certificates: 2017.08.26-h1d4fec5_0      --> 2018.03.07-0     
    certifi:         2017.7.27.1-py36h8b7b77e_0 --> 2018.4.16-py36_0 
    openssl:         1.0.2m-h8cfc7e7_0          --> 1.

In [6]:
cleaned = cleaned.copy()
cleaned.head()

country  date  Access Electricity  Cellular %  \
country     date                                                      
Afghanistan 2016  Afghanistan  2016                 NaN   66.003744   
            2015  Afghanistan  2015                 NaN   61.577682   
            2014  Afghanistan  2014           89.500000   58.845471   
            2013  Afghanistan  2013           75.154373   55.012226   
            2012  Afghanistan  2012           69.100000   51.434547   

                  Fixed broadband %  GDP pcp PPP  Landline %  Secure Servers  \
country     date                                                               
Afghanistan 2016           2.688976  1876.544682    0.348892            49.0   
            2015           2.207969  1861.124332    0.343677            46.0   
            2014           0.479532  1875.447407    0.325861            32.0   
            2013           0.490972  1877.411953    0.313466            30.0   
            2012           0.502942  1873.153946    0.301822            33.0   

                  Urbanisation  internet_users Code      Region cgroup  
country     date                                                        
Afghanistan 2016        27.132       10.595726  AFG  South Asia      L  
            2015        26.703        8.260000  AFG  South Asia      L  
            2014        26.282        7.000000  AFG  South Asia      L  
            2013        25.871        5.900000  AFG  South Asia      L  
            2012        25.468        5.454545  AFG  South Asia      L

# First, getting the example from plotly, only with my data


In [9]:
import plotly.plotly as py
import pandas as pd

In [18]:
map_df = cleaned[cleaned['date'] == 2014][['country', 'Code', 'internet_users']]
map_df.head()

,,country,Code,internet_users
country,date,,,
Afghanistan,2014,Afghanistan,AFG,7.00
Albania,2014,Albania,ALB,60.10
Algeria,2014,Algeria,DZA,29.50
Angola,2014,Angola,AGO,10.20
Antigua and Barbuda,2014,Antigua and Barbuda,ATG,67.78


In [27]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [31]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [44]:
configure_plotly_browser_state()
data = [ dict(
        type = 'choropleth', 
        locations = map_df['Code'], 
        z = map_df['internet_users'], 
        text = map_df['country'], 
        colorscale = "Viridis"              31, 32, 33, 34, 35, 36, 37, 38, 39],
        autocolorscale = False, 
        reversescale = True, 
        marker = dict(
            line = dict(
                color = 'rgb(180, 180, 180)', 
                width = 0.5
            )), 
        colorbar=dict(
            autotick = False, 
            tickprefix = '%',
            title= "Internet Users"), 
        #colorscale='Viridis'
    )]
layout = dict(
    title = "2014 Internet Users in %, Source: World Bank",
    geo = dict(
    showframe=False, 
    showcoastlines = False,
    projection = dict(
        type= "Mercator"
    )
    )
)
fig = dict(data=data, layout = layout)
iplot(fig, validate=False, filename="test")

In [39]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')

data = [ dict(
        type = 'choropleth',
        locations = map_df['Code'], 
        z = map_df['internet_users'], 
        text = map_df['country'], 
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '%',
            title = 'Internet Users'),
      ) ]

layout = dict(
    title = '2014 Internet Users',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot( fig, validate=False, filename='d3-world-map' )